# Exercise 3: Finite size scaling

In this exercise, we will determine the critical temperature and critical exponents of the
Ising model.

a) First, we need finite size data. Since generating the data takes some time, it is useful
to save it to disk. Generate your own finite size data from last weeks program (which
can take some time to get well converged results), and/or download finite size data
provided on the GitLab page. Inspect generate_data.py to find out how the data
is structured and how to load the data. Plot the specific heat CV and magnetic
susceptibility χ.

Recall: 
$$ C_v = \frac{1}{k_BT^2} \left(\left<E^2\right> - \left<E\right>^2\right)  $$

$$ \chi = \frac{1}{k_BT} \left(\left<M^2\right> - \left<M\right>^2\right) $$


In [ ]:
%matplotlib qt5
from ising_model import get_data
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def calc_Cv(E: np.ndarray, T: float, kb: float = 1) -> float:
    return 1/(kb*T**2) * np.var(E)

calc_Cv_vectorized = np.vectorize(calc_Cv, excluded=["E"], signature="(m),()->()")

def calc_chi(M: np.ndarray, kbT: float) -> float:
    return 1/(kbT) * np.var(np.abs(M))

calc_chi_vectorized = np.vectorize(calc_chi, excluded=["M"], signature="(m),()->()")


In [ ]:
Es_20, Ms_20, Ts = get_data(20)

In [ ]:
Cvs_20 = calc_Cv_vectorized(Es_20, Ts)
chis_20 = calc_chi_vectorized(Ms_20, Ts)

In [ ]:
plt.figure()
plt.suptitle("Verification plot")
plt.subplot(1,2,1)
plt.plot(Ts, Cvs_20 / 20**2, label="L=20")
plt.xlabel("T")
plt.legend()
plt.title("Cv")
plt.subplot(1,2,2)
plt.plot(Ts, chis_20, label="L=20")
plt.xlabel("T")
plt.title("$\\chi$")
plt.legend()
plt.show()

b) The specific heat CV and magnetic susceptibility χ have maxima, which move with
increasing system size L. Determine the temperature corresponding to the maxima
for various system sizes and plot them versus $\frac{1}{L}$. Extrapolate to L → ∞ to obtain
an estimate for the critical temperature Tc.
Hint: You can use the functions `np.argmax` and `np.polyfit`.

In [ ]:
# Get data for the rest of the notebook
Ls = np.unique(np.append(np.arange(5, 30), 2**np.arange(2, 8)))
Tcs_Cv = []
Tcs_chi = []
Cvs = np.empty((Ls.size, Ts.size))
chis = np.empty((Ls.size, Ts.size))
for i, L in enumerate(Ls):
    Es, Ms, Ts = get_data(L)
    _Cvs = calc_Cv_vectorized(Es, Ts)
    _chis = calc_chi_vectorized(Ms, Ts)

    Tc_Cv = Ts[np.argmax(_Cvs)]
    Tc_chi = Ts[np.argmax(_chis)]

    Tcs_Cv.append(Tc_Cv)
    Tcs_chi.append(Tc_chi)

    Cvs[i, :] = _Cvs
    chis[i, :] = _chis


In [ ]:
plt.figure()
plt.subplot(1,3,1)
plt.scatter(1/Ls, Tcs_Cv)
plt.xlabel("1/L")
plt.title("Tc from Cv")
plt.subplot(1,3,2)
plt.scatter(1/Ls, Tcs_chi)
plt.xlabel("1/L")
plt.title("Tc from $\\chi$")
plt.subplot(1,3,3)
plt.scatter([*1/Ls, *1/Ls], [*Tcs_Cv, *Tcs_chi])
plt.xlabel("1/L")
plt.title("Tc")
plt.show()

In [ ]:
# The Cv-data cannot really be extrapolated, as seen on the plot above
print("== Tc estimates ==")
print(f" chi  :     {np.polyval(np.polyfit(1/Ls, Tcs_chi, 1), 0) :.3f}")
print(f" Cv   :     {np.polyval(np.polyfit(1/Ls, Tcs_Cv, 1), 0) :.3f}")
print(f" both :     {np.polyval(np.polyfit([*1/Ls, *1/Ls], [*Tcs_Cv, *Tcs_chi], 1), 0) :.3f}")
print(f" exact:     {2 / np.log(1 + np.sqrt(2)) :.3f}")

c) Another quantity which is especially good to obtain the critical temperature is the
so-called Binder cumulant, introduced by Binder in 1981 and defined as

$$ U_B = \frac{3}{2}\left(1 - \frac{\left<M^4\right>}{3\left<M^2\right>^2}\right) $$

Plot the Binder cumulant for various system sizes. Find the crossings of UB between
curves corresponding to L and 2L, and include them into the previous plot from b).

In [ ]:
def binder_cumulant(Ms: np.ndarray) -> float:
    M4 = np.mean(Ms**4)
    M22 = np.mean(Ms**2)**2
    return 1.5 * (1 - M4 / (3*M22))

In [ ]:
Ls_UB = 2**np.arange(2, 8)
UBs = np.empty((Ls_UB.size, Ts.size))

for i, L in enumerate(Ls_UB):
    Es, Ms, Ts = get_data(L)
    for j in range(Ts.size):
        UB = binder_cumulant(Ms[j, :])
        UBs[i, j] = UB

In [ ]:
plt.figure()
plt.title("Binder cumulant")
for i, L in enumerate(Ls_UB):
    plt.plot(Ts, UBs[i], label=f"{L = }")
plt.xlabel("T")
plt.legend()
plt.show()


In [ ]:
# Find intersection by inspection
T_UB = 2.265

Right at the critical temperature TC , the (infinite) system becomes scale invariant. As
one approaches the critical point, different macroscopic quantities scale with a power law
in τ ≡ $\frac{T - T_C}{T_C}$. The exponents of these power laws are universal, i.e., they can coincide for
systems with different microscopic descriptions (which defines the “universality class”).
For example, the correlation length diverges as ξ ∝ |τ |−ν , the specific heat as CV ∝ |τ |−α,
the order parameter in the ordered phases as |M | ∝ (−τ )β , and the susceptibility as
χ ∝ |τ |−γ . The correlation length of a finite system is bounded by the system size L.
From that, one can derive a universal finite size scaling near the critical point.

d) Instead of viewing finite-size effect as a nuisance in cutting off power laws, one
can exploit the dependency of critical exponents on system size L to extract the
exponents. Recall the maxima value of specific heat CV and magnetic susceptibility
χ are cut off by

$$ \chi \sim L^\frac{\gamma}{\nu} $$

$$ C_V \sim L^\frac{\alpha}{\nu} $$

Find out the ratio $\frac{\gamma}{\nu}$ and $\frac{\alpha}{\nu}$.
Hint: You can use the functions `plt.loglog` and `np.polyfit`.

In [ ]:
plt.figure()
plt.loglog(1/Ls, Tcs_chi)
plt.loglog(1/Ls, Tcs_Cv)
plt.show()
print("gamma / nu", np.polyfit(1/Ls, np.log(Tcs_chi), 1)[1])
print("alpha / nu", np.polyfit(1/Ls, np.log(Tcs_Cv), 1)[1])

e) The binder cumulant has the finite size scaling

$$ \Phi_{U_B}\left(\tau L^\frac{1}{\nu}\right) $$


where ΦUB is an unknown, universal function. Plot UB versus $\tau L^\frac{1}{\nu}$ for various L.
Vary the unknown exponent ν until the curves all appear on a single line.

In [ ]:
# Animation to make finding nu easier
from matplotlib.animation import FuncAnimation
from matplotlib.widgets import Slider

fig, (slider_ax, function_ax) = plt.subplots(1, 2)

animation_nu = [1]    # default 1, use list to make updating easier
Tc = 2 / np.log(1 + np.sqrt(2)) # ad hoc
tau = (Ts - Tc) / Tc

nu_slider = Slider(slider_ax, '$\\nu$ ', valmin=0, valmax=5, 
            valinit=animation_nu[0], valfmt='%.2f', facecolor='#cc7000')

def update_nu(_nu):
    animation_nu[0] = _nu
nu_slider.on_changed(update_nu)

function_ax.set_xlabel("$\\tau L^\\frac{1}{\\nu}$")
function_ax.set_ylabel("$U_B$")

plots = []
for i, L in enumerate(Ls_UB):

    plot ,= function_ax.plot(tau * L**(1 / animation_nu[0]), UBs[i])
    plots.append(plot)

def animation(_):
    for i, L in enumerate(Ls_UB):
        plots[i].set_data(tau * L**(1 / animation_nu[0]), UBs[i])
    x = tau * Ls[-1]**(1 / animation_nu[0])
    function_ax.set_xlim(np.min(x)*1.1, np.max(x)*1.1)

ani = FuncAnimation(fig, animation, blit=False, interval=50)

fig.show()

In [ ]:
# By inspection
optimal_nu = animation_nu[0]
optimal_nu

f) The susceptibility and specific heat scale as

$$ \chi = L^\frac{\gamma}{\nu}\Phi_{\chi}\left(\tau L^\frac{1}{\nu}\right) $$


$$ C_V = L^\frac{\alpha}{\nu}\Phi_{C_V}\left(\tau L^\frac{1}{\nu}\right) $$

with other unknown scaling functions Φχ, ΦCV . Try to find the exponents γ and α
by plotting χ/$L^\frac{\gamma}{\nu}$ and CV /L^\frac{\alpha}{\nu}$ versus τ$L^\frac{1}{\nu}$$ and varying the exponents until you get
a data collapse. How well is the hyperscaling relation νd = 2 − α fulfilled?
Hint: While for an exponent α = 0 formally CV ∝ τ −α = const., the leading scaling
behaviour is in this case CV ∝ − log(τ ). For the finite size scaling, this means
CV = $\log(L)\Phi_{C_V}\left(\tau L^\frac{1}{\nu}\right)$

.

In [ ]:
fig, (slider_ax, function_ax) = plt.subplots(1, 2)

animation_gamma = [1]    # default 1, use list to make updating easier
Tc = 2 / np.log(1 + np.sqrt(2)) # ad hoc
tau = (Ts - Tc) / Tc

gamma_slider = Slider(slider_ax, '$\\nu$ ', valmin=0, valmax=5, 
            valinit=animation_nu[0], valfmt='%.2f', facecolor='#cc7000')

def update_gamma(_gamma):
    animation_gamma[0] = _gamma
gamma_slider.on_changed(update_gamma)

function_ax.set_xlabel("$\\tau L^\\frac{1}{\\nu}$")
function_ax.set_ylabel("$\\chi / L^\\frac{\\gamma}{\\nu}$")

function_ax.set_xlim(-20, 20)

plots = []
for i, L in enumerate(Ls):

    plot ,= function_ax.plot(tau * L**(1 / optimal_nu), chis[i] / L**(animation_gamma[0] / optimal_nu))
    plots.append(plot)

def animation(_):
    for i, L in enumerate(Ls):
        plots[i].set_data(tau * L**(1 / optimal_nu), chis[i] / L**(animation_gamma[0] / optimal_nu))
    y = chis[i] / Ls[-1]**(animation_gamma[0] / optimal_nu)
    function_ax.set_ylim(np.min(y)*1.1, np.max(y)*1.1)

ani = FuncAnimation(fig, animation, blit=False, interval=50)

fig.show()

In [ ]:
optimal_gamma = animation_gamma[0]
optimal_gamma

In [ ]:
fig, (slider_ax, function_ax) = plt.subplots(1, 2)

animation_alpha = [1]    # default 1, use list to make updating easier
Tc = 2 / np.log(1 + np.sqrt(2)) # ad hoc
tau = (Ts - Tc) / Tc

alpha_slider = Slider(slider_ax, '$\\nu$ ', valmin=0, valmax=5, 
            valinit=animation_nu[0], valfmt='%.2f', facecolor='#cc7000')

def update_alpha(_alpha):
    animation_alpha[0] = _alpha
alpha_slider.on_changed(update_alpha)

function_ax.set_xlabel("$\\tau L^\\frac{1}{\\nu}$")
function_ax.set_ylabel("$Cv / L^\\frac{\\alpha}{\\nu}$")

function_ax.set_xlim(-20, 20)

plots = []
for i, L in enumerate(Ls):

    plot ,= function_ax.plot(tau * L**(1 / optimal_nu), Cvs[i] / L**(animation_alpha[0] / optimal_nu))
    plots.append(plot)

def animation(_):
    for i, L in enumerate(Ls):
        plots[i].set_data(tau * L**(1 / optimal_nu), Cvs[i] / L**(animation_alpha[0] / optimal_nu))
    y = Cvs[i] / Ls[-1]**(animation_alpha[0] / optimal_nu)
    function_ax.set_ylim(np.min(y)*1.1, np.max(y)*1.1)

ani = FuncAnimation(fig, animation, blit=False, interval=50)

fig.show()

In [ ]:
optimal_alpha = animation_alpha[0]
optimal_alpha

g) Modify the program generating the Monte carlo data to simulate the (ferromagnetic)
Ising model on a triangular lattice. (For the provided program, this requires only to
add 2 lines (and maybe changing output filename)!). Find the critical temperature.
Are the critical exponents the same as on the square lattice, i.e., are these two
models in the same universality class?

In [ ]:
# I re-ran the entire notebook, where I imported the triangular model instead of the square one.
# The results were as follows:
